In [1]:
pinecone_api_key = "YOUR_API_KEY"

In [2]:
from langchain_community.retrievers import PineconeHybridSearchRetriever
import os
from pinecone import Pinecone, ServerlessSpec

In [3]:
index_name = "hybrid-search-langchain-pinecone"
## initialize pinecone client
pc = Pinecone(api_key=api_key)

# create the index
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384, # dimension of dense vector
        metric='dotproduct',  # sparse values supported only for dotproduct
        spec=ServerlessSpec(cloud='aws',region="us-east-1")
    )

In [4]:
index=pc.Index(index_name)
index

In [7]:
pip install -U langchain-huggingface

  Using cached tokenizers-0.21.0-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
Using cached tokenizers-0.21.0-cp39-abi3-win_amd64.whl (2.4 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\users\kriti\appdata\local\programs\python\python311\lib\site-packages\face_recognition-1.3.0-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chromadb 0.5.23 requires tokenizers<=0.20.3,>=0.13.2, but you have tokenizers 0.21.0 which is incompatible.


In [8]:
## vector embedding and sparse matrix
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\kriti\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kriti\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [9]:
from pinecone_text.sparse import BM25Encoder

bm25_encoder = BM25Encoder().default()
bm25_encoder

100% [....................................] 65406227 / 65406227

In [10]:
sentences=[
    "In 2023, I visited Paris",
    "In 2022, I visited Mew York",
    "In 2021, I visited New Orleans"
]

# tfidf values on these sentence
bm25_encoder.fit(sentences)

# store the values to a json file
bm25_encoder.dump("bm25_values.json")

# load to your BM25Encoder object
bm25_encoder = BM25Encoder().load("bm25_values.json")

  0%|          | 0/3 [00:00<?, ?it/s]

In [12]:
retriever = PineconeHybridSearchRetriever(embeddings=embeddings,sparse_encoder=bm25_encoder,index=index)

In [13]:
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x0000013D02126290>, index=<pinecone.data.index.Index object at 0x0000013D1C7F5F90>)

In [15]:
retriever.add_texts(
    [
    "In 2023, I visited Paris",
    "In 2022, I visited Mew York",
    "In 2021, I visited New Orleans"
    ]
)

  0%|          | 0/1 [00:00<?, ?it/s]

In [16]:
retriever.invoke("What city did I visit last")

[Document(metadata={'score': 0.271121591}, page_content='In 2021, I visited New Orleans'),
 Document(metadata={'score': 0.237657696}, page_content='In 2022, I visited Mew York'),
 Document(metadata={'score': 0.23593007}, page_content='In 2023, I visited Paris')]